In [1]:
import os
import tkinter
from tkinter import filedialog
import numpy as np
from numpy.fft import fft2, ifft2, fftshift, ifftshift
from scipy import misc
from scipy import ndimage
import cv2
import math

In [2]:
# Define functions here
def search_for_file_path(prompt):

    root = tkinter.Tk()
    root.withdraw()  # use to hide tkinter window
    currdir = os.getcwd()
    file_dir = filedialog.askdirectory(
        parent=root,
        initialdir=currdir,
        title=prompt,
        mustexist=True,
    )

    return file_dir

In [3]:
img_dir = search_for_file_path('Select Images Folder')

In [4]:
imgs = os.listdir(img_dir)

In [5]:
atticRoom = cv2.imread(os.path.join(img_dir, imgs[0]),0)
bathroom = cv2.imread(os.path.join(img_dir, imgs[1]),0)

In [6]:
def show_image(input_image):
    cv2.imshow('Displaying Image...', input_image)
    cv2.waitKey(0) 
    cv2.destroyAllWindows() 

In [9]:
# define image parameters
n,m = np.shape(atticRoom)
numRows = n
numCols = m

# Calculate gaussian filter sigma by cycles/image cutoff (when amplitude gain = 0.5)
def gaussian_filter_sigma(cycles_per_img_cutoff, amplitude_gain=0.5):
    numer = -1.0*(cycles_per_img_cutoff**2)
    denom = 2 * np.log(amplitude_gain)
    return math.sqrt(numer/denom)

sigma8 = gaussian_filter_sigma(8)
sigma16 = gaussian_filter_sigma(16)
sigma24 = gaussian_filter_sigma(24)

In [14]:
# gaussian filter (in fourier space)
def makeGaussianFilter(numRows, numCols, sigma, highPass=True):
    centerI = int(numRows/2) + 1 if numRows % 2 == 1 else int(numRows/2)
    centerJ = int(numCols/2) + 1 if numCols % 2 == 1 else int(numCols/2)
    def gaussian(i,j):
        coefficient = math.exp(-1.0 * ((i - centerI)**2 + (j - centerJ)**2) / (2 * sigma**2))
        return 1 - coefficient if highPass else coefficient
 
    return np.array([[gaussian(i,j) for j in range(numCols)] for i in range(numRows)])

In [15]:
def filterDFT(imageMatrix, filterMatrix):
    shiftedDFT = fftshift(fft2(imageMatrix))
    filteredDFT = shiftedDFT * filterMatrix
    return ifft2(ifftshift(filteredDFT))

In [26]:
def hybridImage(highFreqImg, lowFreqImg, sigmaHigh, sigmaLow):
    highPassed = highPass(highFreqImg, sigmaHigh)
    lowPassed = lowPass(lowFreqImg, sigmaLow)
    return highPassed + lowPassed

In [73]:
highPassed = filterDFT(bathroom, makeGaussianFilter(n, m, sigma24, True))
lowPassed = filterDFT(atticRoom, makeGaussianFilter(n, m, sigma8, False))

In [74]:
cv2.imwrite('hybrid.png', abs((highPassed+lowPassed)/2.0))

True

In [46]:
fourier_filter = makeGaussianFilter(n, m, sigma24, True)
np.shape(fftshift(fft2(atticRoom)))

(768, 1024)

In [67]:
uint_img = np.array(abs(highPassed)).astype('uint8')
grayImage = cv2.cvtColor(uint_img, cv2.COLOR_GRAY2BGR)

In [65]:
def convert_to_gray(img_mat):
    img_mat = img_mat/np.max(img_mat)
    uint_img = np.array(img_mat*255).astype('uint8')
    grayImage = cv2.cvtColor(uint_img, cv2.COLOR_GRAY2BGR)
    return grayImage

In [70]:
cv2.imwrite('notgrayImage.png', abs(highPassed))

True

In [78]:
test = filterDFT(bathroom, makeGaussianFilter(n, m, 100, True))
show_image(convert_to_gray(abs(test/(1024*768))))